In [ ]:
# hey there! welcome to this interactive cuda notebook

# this is meant to be a hands-on companion to the article at https://siboehm.com/articles/22/CUDA-MMM
# if you haven't read it yet, thats fine! its probably good to read along with the exercises though

# we're gonna dive into some cuda optimizations, starting with a basic implementation and working our way up to some pretty cool tricks.
# you'll get to write actual cuda code and see how different techniques can boost performance

# here's what you should probably know:

# prerequisites:
# - basic c++ (writing a for loop)
# - basic python
# - a bit of linear algebra (knowing what a matrix multiplication is)
# - curiosity about making gpus go fast

# feel free to google/read the article/ask a language model as you go if you get stuck
# or provide the author of this notebook feedback on sections that confused you (ideal)

# lets get some setup out of the way

# Setup

In [ ]:
!nvcc --version # make sure this works! this code will not work if you are using a CPU (you can use a T4 in colab)

In [ ]:
# run this cell for setup

# clones repo to get the other files
!git clone https://github.com/arb8020/optimizing-matmul-exercises
%cd optimizing-matmul-exercises

# pip
!pip install requests numpy

import sys
import os

src_path = os.path.abspath('src')
if src_path not in sys.path:
    sys.path.append(src_path)

from helper_functions import check_solution, compile_and_run_kernel

!cp src/test_sgemm.cu .
!cp src/test_sgemm2.cu .
!cp src/test_sgemm3.cu .

!nvcc src/test_sgemm.cu -c -o test_sgemm.o
!nvcc src/test_sgemm2.cu -c -o test_sgemm2.o
!nvcc src/test_sgemm3.cu -c -o test_sgemm3.o

# Kernel 1: Naive Implementation

### Introduction

In [ ]:
""" CUDA Hierarchy """

# a CUDA kernel is an operation to be run on the GPU
# Computation in CUDA is split into three levels: grid, block, thread
# creating a kernel creates a grid, made up of blocks
# each of these blocks has a bunch of threads which share memory

# gridDim contains the amount of blocks in a grid
# gridDim is a 3 size vector
# gridDim.x, gridDim.y, gridDim.z

# blockDim contains the amount of threads in the block
# blockDim is a 3 size vector
# blockDim.x, blockDim.y, blockDim.z
# blocks can have up to 1024 threads

In [ ]:
"""SGEMM"""

# single precision
# generalized
# matrix multiplication

# basic matrix multiplication
# C = AB
# generalized version allows us to easily linearly transform this
# C = alpha * (AB) + beta * (C)
# note that if alpha = 1, beta = 0, it simplifies to the basic form

# A is of dimension M x K
# B is of dimension K x N
# producing matrix C of dimension M x N
# each entry of C is the dot product of row in A and col in B

# so if we iterate through K, i = 0 ... K
# we compute the row for A as row * K + i
# and compute the column for B as i * N + col

# finally, we scale each index of C by alpha and beta

In [ ]:
""" Naive Kernel Implementation """
# CUDA kernels are written from the perspective of a single thread
# for example, if we write:
# b[i] = a[i] + 10
# it will do this function for all threads i

# for our first kernel, we will let each thread compute an entry
# to compute the right value, each thread needs to get the right index value
# since we're in 2D, each thread needs to know:

# which x and y block of threads its in
# how big the blocks of threads are (x and y dims)
# thread index inside the block (x and y axis)

# say we have a matrix of size 5 x 5
# a block size of 3 x 3 (9 threads per block)
# and a 3 x 3 grid (could load matrix size 9 x 9)
# it looks something like:
# grid: [block_0], [block_1], ...
# block: [thread_0, thread_1, ...]

# example:
# x: [0, 1, 2], [3, 4, 5], [6, 7, 8]
# y: [0, 1, 2], [3, 4, 5], [6, 7, 8]

# so the thread (3,7) would be

# x = 3 -> thread_0 in block 1
# y = 7 -> thread_1 in block 2

# note that since the matrix is of size 5 x 5 in this example
# we would need to check if the current thread is even used
# in this example, y = 7 > 4 so we don't actually use that thread

### Code

In [ ]:
# .cu doesnt have syntax highlighting so we write it as a .cpp and then just change name later

In [ ]:
%%writefile kernel_1.cpp
#include <iostream>
#include <cmath>
#include "test_sgemm.cu"

// CUDA kernel function for performing a naive implementation of GEMM (sgemm)
// STUDENT
__global__ void sgemm_naive(int M, int N, int K, float alpha, const float *A, const float *B, float beta, float *C) {
    // think about the CUDA hierarchy


    // calculate the x-coordinate of the current thread within the grid
    // we will need to add threadIdx to some block offset
    // we can probably use blockIdx, blockDim to determine the offset
    uint x_block_offset = ...
    const uint x = ...
    // calculate the y-coordinate of the current thread within the grid
    uint y_block_offset = ...
    const uint y = ...

    // check if the current thread's coordinates are within the bounds of the output matrix C
    // this is known as a 'guard'
    // we'll likely need to compare the above variables with the respective matrix dimensions
    // TODO: fill out the if statement
    if (condition) {
        // initialize a temporary variable to store the dot product
        float tmp = 0.0;

        // compute the dot product of the x-th row of matrix A and the y-th column of matrix B
        // TODO: add the correct stop/increment condition for the for loop
        for (int i = 0; cond; ++1) {
            // TODO: accumulate results of matrix dot product into tmp
            // remember the dot product formula!
            // A: current row * A_columns + loop counter
            // B: loop counter * B_columns + current column
            tmp += ...
        }

        // compute the final value for the element at position (x, y) in the output matrix C
        // by multiplying the temporary result 'tmp' by 'alpha', adding the product of 'beta'
        // and the existing value at that position in C, and storing the result in C

        // TODO: compute the final value as described above
        // make sure to write it into the correct index as well!
        C[] = ...
    }
}

int main() {
    int M = 512, N = 512, K = 512; // MNK will always be equal

    std::cout << "testing naive SGEMM: " << std::endl;

    dim3 gridDim(CEIL_DIV(M, 32), CEIL_DIV(N, 32), 1);
    dim3 blockDim(32, 32, 1);

    test_sgemm(M, N, K, gridDim, blockDim, sgemm_naive);

    return 0;
}

In [ ]:
# this block will compile and run your code!
!cp kernel_1.cpp kernel_1.cu
!nvcc kernel_1.cu -o kernel1 -lcublas
!./kernel1

In [ ]:
# this block will check your solution
with open('kernel_1.cu', 'r') as file:
    user_kernel_1 = file.read()
check_solution(1, user_kernel_1, M=512, N=512, K=512)

# Roofline Model:

### Introduction

In [ ]:
""" Motivating Improvements: Roofline Model """
# the underlying concept that motivates the improvements in this article is the roofline model
# roofline model: https://en.wikipedia.org/wiki/Roofline_model
# the Roofline model helps us determine performance bottlenecks
# on the X axis we have Arithmetic Intensity:
# Arithmetic Intensity is the ratio of floating point operations to the amount of data transferred
# FLOP/byte
# on the Y axis we have Memory Bandwidth:
# the rate of data transfer between memory and processor
# Byte/s
# our performance is FLOP/s - which we can plot as a function of our MB and AI
# if we have high AI, we are doing a lot of computations per unit of data: compute bound
# if we have high MB, we are getting data as fast as possible, likely from registers or L1 cache vs main memory
# so the goal is to move towards higher AI by optimizing our operations per load
# and move towards peak memory bandwidth by having as much data as close to the processor as possible

# in the below code, we'll explore the theoretical upper bound that we're shooting for with our kernel

### Code

In [ ]:
def calculate_roofline(m, n, k, memory_bandwidth, computational_performance, data_type_size):

    operations = None  # calculate number of operations
    loads = None  # calculate number of loads
    stores = None  # calculate number of stores

    data_transfer = None # total amount of data to transfer

    computation_time = None  # time to perform operations

    memory_bandwidth_bytes = None  # convert to bytes/second
    data_transfer_time = None  # time to transfer data

    # in the lower bound, we can imagine that GPUs are able to start processing data as soon as part of it loads in
    # so the latency is only bounded by either compute time or data transfer time - whichever is larger
    lower_bound_latency = None

    # calculate performance (operations/time)
    # make sure to convert back to gigaflops
    gflops = None

    return {
        'operations': operations,
        'data_transfer': data_transfer,
        'computation_time': computation_time,
        'data_transfer_time': data_transfer_time,
        'lower_bound_latency': lower_bound_latency,
        'gflops': gflops
    }



# NVIDIA T4 GPU specifications
# https://www.nvidia.com/content/dam/en-zz/Solutions/Data-Center/tesla-t4/t4-tensor-core-datasheet-951643.pdf
# note that we are using fp16
memory_bandwidth = None  # do this in GB/s
computational_performance = None  # TFLOPs
data_type_size = None  # bytes (for float16)

# test your implementation with different matrix dimensions
# what upper bound do you find, and why?
m, n, k = 512, 512, 512

result = calculate_roofline(m, n, k, memory_bandwidth, computational_performance, data_type_size)

print("Roofline Model Results:")
print(f"Total operations: {result['operations']:,}")
print(f"Computation time: {result['computation_time']:.6f} seconds")
print(f"Data transfer time: {result['data_transfer_time']:.6f} seconds")
print(f"Lower bound latency: {result['lower_bound_latency']:.6f} seconds")
print(f"Performance: {result['gflops']:.2f} GFLOPS")

# check your solution
from helper_functions import check_roofline_calculation

check_roofline_calculation(calculate_roofline)

# experiment with different matrix sizes or hardware specifications!
# try changing the values of m, n, k, memory_bandwidth, or computational_performance
# and run the calculation again to see how it affects the results

### visuals

In [ ]:
# here, we can visualize the roofline model and confirm the calculations we made before
# we may not be able to hit the peak value, but we can certainly try!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# matrix dimensions
m_values = [2**x for x in range(12)]  # test different values of m
n = 512  # constant value for n
k = 1024  # constant value for k

# NVIDIA T4 GPU specifications
memory_bandwidth = 320  # GB/s
computational_performance = 65  # TFLOPs

# data type
data_type = np.float16
data_type_size = np.dtype(data_type).itemsize  # size of the data type in bytes

# lists to store the results
arithmetic_intensity_values = []
gflops_values = []

for m in m_values:
    # calculate the total number of operations
    operations = 2 * m * k * n

    # calculate the total amount of data transferred
    data_transfer = (m * k + k * n + m * n) * data_type_size

    # calculate arithmetic intensity (operations per byte)
    arithmetic_intensity = operations / data_transfer
    arithmetic_intensity_values.append(arithmetic_intensity)

    # calculate the computation time
    computation_time = operations / (computational_performance * 1e12)

    # calculate the data transfer time
    memory_bandwidth_bytes = memory_bandwidth * 1e9  # Convert to bytes/second
    data_transfer_time = data_transfer / memory_bandwidth_bytes

    # lower bound on latency: from latency hiding
    lower_bound_latency = max(computation_time, data_transfer_time)

    # calculate throughput
    gflops = operations / (lower_bound_latency * 1e9)
    gflops_values.append(gflops)

# plot the results
plt.figure(figsize=(8, 6))
plt.plot(arithmetic_intensity_values, gflops_values, marker='o', label='Theoretical GFLOPS')

# plot memory bandwidth limit
ai_values = np.linspace(min(arithmetic_intensity_values), max(arithmetic_intensity_values), 100)
bw_limit = memory_bandwidth * ai_values
plt.plot(ai_values, bw_limit, linestyle='--', label='Memory Bandwidth Limit')

# Plot computational performance limit
comp_limit = [computational_performance * 1e3] * len(ai_values)
plt.plot(ai_values, comp_limit, linestyle='--', label='Computational Performance Limit')

plt.xlabel('Arithmetic Intensity (FLOP/byte)')
plt.ylabel('Throughput (GFLOPS)')
plt.title('Roofline Model')
plt.grid(True)
plt.legend()
plt.show()

# Kernel 2: Global Memory Coalescing and Warps


### Introduction

In [ ]:
""" Warps """
# to get better memory accesses: we need to talk about warps
# warps are groups of 32 threads next to each other
# these are the smallest unit of compute that get instructions together
# they are instructed by a warp scheduler

# warp schedulers control consecutive thread ids
# so we might have
# warp_0 : t_0 ... t_31
# warp_1 : t_32 ... t_63

# the overall consecutive thread id is computed as follows:
# threadId = threadIdx.x + blockDim.x * (threadIdx.y + blockDim.y * threadIdx.z)

# say our threads were accessing sequential memory
# t_0 accesses address 0, t_1 gets address 1, etc
# threads that are part of the same warp will get memory accesses all together
# this is known as 'global memory coalescing'
# note that the accesses can be out of order as long as the memory is contiguous
# t_0 accessing 0, t_1 accessing 2, t_2 accessing 1 can get coalesced
# this is kind of similar to cache line optimizations from CPU programming

# we will try to take advantage of global memory coalescing below
# by introducing a BLOCKSIZE variable

### Code

In [ ]:
%%writefile kernel_2.cpp
#include <iostream>
#include <cmath>
#include "test_sgemm.cu"

// added BLOCKSIZE as a constant
// this will be 32: the amount of threads in a warp
template <const uint BLOCKSIZE>
__global__ void sgemm_coalesce(int M, int N, int K, float alpha, const float *A, const float *B, float beta, float *C) {


    // change variable names to reflect we are trying to use rows/columns
    // rather than generic x, y indices

    // use the calculation for consecutive threadId to derive an index
    // which takes advantage of global memory coalescing

    // remember that the overall consecutive thread id is computed as follows:
    // threadId = threadIdx.x + blockDim.x * (threadIdx.y + blockDim.y * threadIdx.z)
    // also remember that we want to access memory within the same warp: threads 0-31

    // the BLOCKSIZE constant may come in handy here
    // further hints:
    // we probably want to replace blockDim with BLOCKSIZE (# threads in a warp)
    // for rows, we likely need to do division
    // for columns we likely need to do modulo
    const int cRow = ...
    const int cCol = ...

    // this is the same as the code from before
    if (cRow < M && cCol < N) {
      float tmp = 0.0;
      for (int i = 0; i < K; ++i) {
        tmp += A[cRow * K + i] * B[i * N + cCol];
      }
      C[cRow * N + cCol] = alpha * tmp + beta * C[cRow * N + cCol];
    }

}

int main() {
    int M = 512, N = 512, K = 512;

    std::cout << "testing SGEMM with Coalescing: " << std::endl;

    // make blockDim one dimensional
    dim3 gridDim(CEIL_DIV(M, 32), CEIL_DIV(N, 32));
    dim3 blockDim(32 * 32);

    test_sgemm(M, N, K, gridDim, blockDim, sgemm_coalesce<32>);

    return 0;
}

In [ ]:
!cp kernel_2.cpp kernel_2.cu

!nvcc kernel_1.cu -o kernel1 -lcublas
!./kernel1
!nvcc kernel_2.cu -o kernel_2 -lcublas
!./kernel_2

In [ ]:
# this block will check your solution
with open('kernel_2.cu', 'r') as file:
    user_kernel_2 = file.read()
check_solution(2, user_kernel_2, M=512, N=512, K=512)

# Kernel 3: Shared Memory


### Introduction

In [ ]:
""" Memory Hierarchy """
# Let's spell out the Memory Hierarchy of a GPU:
# understanding this will help us increase our Memory Bandwidth
# DRAM: very big, but very slow
# L2 Cache: 1000x smaller, but way faster (not on chip)
# SM: streaming multiprocessor: each block of threads gets executed by one
# each SM gets SMEM, shared memory on chip (kinda like L1 cache), and registers

# to optimize for Byte/s we want our code to use
# registers > SMEM > L2 > DRAM

# so rather than reading from L2/DRAM (global memory) every time
# we can instead load chunks of A's row and B's column
# we'll choose a chunk size that fits onto the SMEM
# this will allow us to compute and add to C more efficiently
# since the values we need will be in almost the closest possible access point

# note that when we use shared memory we need to avoid race conditions
# if some threads start running too far ahead of others
# we might write to memory that threadA needed to read before threadB wrote to it
# CUDA provides a function called __syncthreads()
# __syncthreads() effectively stops all the threads at that line of code until the other threads get there
# this is important especially when loading from shared memory
# so that the threads won't start working until all other threads have finished loading into shared memory

### Code

In [ ]:
%%writefile kernel_3.cpp
#include <iostream>
#include <cmath>
#include "test_sgemm.cu"

template <const uint BLOCKSIZE>
__global__ void sgemm_shared(int M, int N, int K, float alpha, const float *A, const float *B, float beta, float *C) {

  // we basically want to break this index from above down into smaller parts
  // so that we can load chunks into shared memory
  // const int cRow = blockIdx.x * BLOCKSIZE + (threadIdx.x / BLOCKSIZE);
  // const int cCol = blockIdx.y * BLOCKSIZE + (threadIdx.x % BLOCKSIZE);

  // first get output block row and col (first part)
  // TODO: first part of the index calculation
  const uint cRow = blockIdx.x;
  const uint cCol = blockIdx.y;

  // get the inner row and column
  // TODO: second part of the index calculation
  const uint threadCol = threadIdx.x % BLOCKSIZE;
  const uint threadRow = threadIdx.x / BLOCKSIZE;

  // move pointers in A, B, C to the right spots
  // we want A such that row = cRow, col = 0
  // we want B such that row = 0, col = cCol
  // we want C such that row = cRow, col = cCol

  // we'll need to incorporate:
  // blocksize (size of matrix we are going to load in)
  // dimension of matrix A, B
  A += ...
  B += ...
  C += ...

  // allocate shared memory for the current block
  // we would need the total size of the matrix: rows * columns
  // in this case they should both be blocksize

  __shared__ float As[BLOCKSIZE * BLOCKSIZE];
  // TODO: write the shared memory allocation for B

  // temporary float
  float tmp = 0.0;

  // here we iterate through the larger matrix, and actually load in the chunks
  // we prob don't want blockIdx to be larger than the shared dimension
  // and we probably should iterate in steps of blocksize
  // TODO: fill in for loop
  for (int blockIdx; blockIdx_comparison; blockIdx_increment) {

    // load chunk into shared memory for A and B
    // the index to load into is likely a function of
    // threadRow, threadCol and BLOCKSIZE (since the shared memory space used blocksize)

    // the index to load from is likely similar
    // it should be a function of the desired row, number of columns, and desired column
    As[] = A[];
    Bs[] = B[];

    // we don't want to move on until all the shared memory has been loaded
    // how can we do that?
    // TODO: prevent race conditions here

    // TODO: compute dot product for the specified block
    for (int dotIdx; dotIdx_comparison; dotIdx_increment) {
      // TODO: fill in the indices
      // this should look pretty similar to kernel 1
      // A: row * multiplicative offset + index
      // B: index * multiplicative offset + col
      tmp += As[] * Bs[];
    }

    // TODO: move A, B pointers
    // this probably uses the blocksize

    A += ...
    B += ...

    // TODO: avoid fetching the next block into the cache before slower threads are done


  }


  // TODO: fill in the indices to correctly iterate through C
  // and do the affine transformation
  C[] = alpha * tmp + beta * C[];

}

int main() {
    int M = 512, N = 512, K = 512;

    std::cout << "testing SGEMM with Shared Memory: " << std::endl;

    // make blockDim one dimensional
    dim3 gridDim(CEIL_DIV(M, 32), CEIL_DIV(N, 32));
    dim3 blockDim(32 * 32);

    test_sgemm(M, N, K, gridDim, blockDim, sgemm_shared<32>);

    return 0;
}

In [ ]:
!cp kernel_3.cpp kernel_3.cu
!nvcc kernel_3.cu -o kernel_3 -lcublas
!./kernel_3

In [ ]:
with open('kernel_3.cu', 'r') as file:
    user_kernel_3 = file.read()
check_solution(3, user_kernel_3, M=512, N=512, K=512)

# Kernel 4: 1D Blocktiling for Calculating Multiple Results per Thread

### Introduction

In [ ]:
""" Increasing Arithmetic Intensity """
# while we managed to move everything into shared memory
# profiling would show that our calculations are stalling
# this is because we are waiting on SMEM accesses
# while SMEM accesses are fast, registers are still best
# so we want our threads to use more register memory
# right now each thread only computes 1 cell, but a register can hold more data than that

# if we have threads use register memory more, we won't wait on SMEM accesses as much
# to have threads use more register memory, each thread should compute more than just 1 cell

# if you remember arithmetic intensity from the roofline model portion: this helps with that!
# rather than calculating 1 result per thread
# (loads 7 values from A and B, loads/stores 1 value into C -> 15 loads, 1 store per result)
# we can compute 4 results per thread
# (loads 14 values from A and B, 4 load/stores into C -> 8 loads, 1 store per result)

# we will start by calculating a column of results at a time: 1D Blocktiling

### Code

In [ ]:
%%writefile kernel_4.cpp
#include <iostream>
#include <cmath>
#include "test_sgemm.cu"

// instead of using a flat blocksize
// we define
// BM: blocksize for dim M (A)
// BN: blocksize for dim N (B)
// BK: blocksize for dim K (A and B)
// TM: thread mult factor (# entries calc per thread)

template <const int BM, const int BN, const int BK, const int TM>
__global__ void sgemm_blocktiling_1d(int M, int N, int K, float alpha, const float *A, const float *B, float beta, float *C) {

  const uint cRow = blockIdx.x;
  const uint cCol =blockIdx.y;

  // each warp used to calculate 32 elements (1 per thread)
  // now we want to calculate 32 * thread_mult_factor (# entries per thread)
  // so we need to adjust threadCol, threadRow to account for this
  // we'll need to access the blockSize for the relevant dimension
  // TODO: fill in threadCol and threadRow
  const int threadCol = ...
  const int threadRow = ...

  // allocate shared memory using blocksizes for A and B
  // again, we will need to use the respective blocksizes
  // TODO: fill it the correct size of array to allocate
  __shared__ float As[];
  __shared__ float Bs[];

  // move pointers similarly to above
  // TODO: use BM, BN, BK instead of the flat blocksize
  A += ...
  B += ...
  C += ...

  // we are making another inner loop
  // so we will need to know where we are inside of A and B
  // for warp-level GMEM coalescing
  // TODO: determine the inner row/column we will need
  const uint innerColA = ...
  const uint innerRowA = ...
  const uint innerColB = ...
  const uint innerRowB = ...

  // before we stored a scalar
  // now since we are doing multiple entries we must store more values
  float threadResults[TM] = {0.0};

  // TODO: replace with the appropriate variable instead of BLOCKSIZE
  for (int blockIdx = 0; blockIdx < K; blockIdx += BLOCKSIZE) {
    // load chunks into shared memory as before
    // remember to prevent against race conditions
    // TODO: use innerRow/Col instead of threadRow/Col

    As[] = A[];
    Bs[] = B[];


    // dot product again
    // TODO: replace BLOCKSIZE
    for (uint dotIdx = 0; dotIdx < BLOCKSIZE; ++dotIdx) {
      // we can store the B value that is needed for all of our computations here

      // TODO: fill out what tmpB initializes to: use the correct index
      float tmpB = Bs[];

      // remember that we are now doing multiple operations per thread
      // we'll probably need TM (thread multiplication factor) somewhere
      // TODO: fill in the for loop to add partial results from A
      for (uint resultIdx = 0; resultIdxcomparison; resultIdxincrement) {
        // how will we compute the correct index to use from As?
        // we need to combine the logic of iterating multiple rows
        // with the dotproduct logic

        // TODO: use the correct index for As to compute the dot product for this column
        threadResults[resultIdx] += As[] * tmpB;
      }

    }

    // move A, B pointer to next tile
    // TODO: replace BLOCKSIZE
    A += BLOCKSIZE;
    B += BLOCKSIZE * N;

    // remember to syncthreads

  }

  // generalize for C again
  // need to do this multiple times per thread now
  for (uint resultIdx = 0; resultIdxcomparison; resultIdxincrement) {
    // similar to the logic from As above
    // we need to combine the usual logic with computing a column for each thread
    // TODO: use the correct index
    C[] = alpha * threadResults[resultIdx] + beta * C[];
  }


}

int main() {
    int M = 4096, N = 4096, K = 4096;

    std::cout << "testing SGEMM with 1D Blocktiling: " << std::endl;

    const uint BM = 64;
    const uint BN = 64;
    const uint BK = 8;
    const uint TM = 8;

    dim3 gridDim(CEIL_DIV(N, BN), CEIL_DIV(M, BM));
    dim3 blockDim((BM * BN) / TM);

    test_sgemm(M, N, K, gridDim, blockDim, sgemm_blocktiling_1d<BM, BN, BK, TM>);

    return 0;
}

In [ ]:
!cp kernel_4.cpp kernel_4.cu
!nvcc kernel_4.cu -o kernel_4 -lcublas
!./kernel_4

In [ ]:
with open('kernel_4.cu', 'r') as file:
    user_kernel_4 = file.read()
check_solution(4, user_kernel_4, M=4096, N=4096, K=4096)

# Kernel 5: Increasing Arithmetic Intensity via 2D Blocktiling


### Introduction

In [ ]:
# here, we expand upon the previous kernel, moving to calculating a 2D tile per thread
# rather than just a column, we compute a tile of rows and columns
# this further increases our arithmetic intensity

### Code

In [ ]:
%%writefile kernel_5.cpp
#include <iostream>
#include <cmath>
#include <cassert>

#include "test_sgemm2.cu"


// we add TN: the amount of rows we want to compute per thread in addition to TM
template <const int BM, const int BN, const int BK, const int TM, const int TN>
__global__ void __launch_bounds__((BM * BN) / (TM * TN), 1) sgemm_blocktiling_2d(int M, int N, int K, float alpha, const float *A, const float *B, float beta, float *C) {

  // as before
  const uint cRow = blockIdx.x;
  const uint cCol = blockIdx.y;

  // our threads are now doing a whole 2D tile worth of work
  const uint totalResultsBlocktile = BM * BN;

  // threads per blocktile
  // this is because we are
  const uint numThreadsBlocktile = totalResultsBlocktile / (TM * TN);

  // we need to adjust this number by the amount of threads we use
  // TODO: use BN and TN to get the correct threadCol/threadRow
  const int threadCol = ...
  const int threadRow = ...


  // allocate space as before
  __shared__ float As[BM * BK];
  __shared__ float Bs[BK * BN];

  // move pointers as before
  A += cRow * BM * K;
  B += cCol * BN;
  C += cRow * BM * N + cCol * BN;

  // inner loop as before
  const uint innerColA = threadIdx.x % BK;
  const uint innerRowA = threadIdx.x / BK;
  const uint innerColB = threadIdx.x % BN;
  const uint innerRowB = threadIdx.x / BN;

  // as before, we need to allocate cache for the grid of results
  float threadResults[TM * TN] = {0.0};

  // similar to tmpB from before
  // we now need to allocate space to store temporary results in each thread
  float regM[TM] = {0.0};
  float regN[TN] = {0.0};

  // loop over block tiles
  for (uint blockIdx = 0; blockIdx < K; blockIdx += BK) {

    // load chunks incrementing by stride, within the size BM/BN
    // we now need to load chunks for each thread, taking into consideration both rows and columns

    // these should technically be outside the loop but they make more logical sense here
    // for implementation hints
    const uint strideA = ...
    const uint strideB = ...

    // TODO: use strideA, strideB and BM/BN
    // to figure out how to load a chunk of data from A/B for each thread
    for (uint loadOffset; loadOffsetComparison; loadOffsetIncrement) {
      // this should look pretty similar to how we loaded in
      // recall As[innerRowA * BK + innerColA] = A[innerRowA * K + innerColA];
      // since we are now loading a chunk of rows instead of just one
      // we need to add a bit more math to this index
      // TODO: determine the indices to load from/into
      As[] = A[];
    }
    // TODO: do the same for B

    // syncthreads
    __syncthreads()

    // advance blocktile as before
    A += BK;
    B += BK * N;

    // modify inner loop for dot product to be 2d logic rather than 1d
    // we're basically doing kernel 1 here for each thread
    for (uint dotIdx = 0; dotIdxComparison; dotIdxIncrement) {
      // TODO: load data into registers from A shared
      for (uint i = 0; iComparison; iIncrement) {
        // TODO: compute index to read from
        // again, the indexing should look pretty familiar
        regM[i] = As[];
      }
      // repeat logic for regN, Bs

      // accumulate into threadResults
      // TODO: expand the previous kernel from 1d to 2d - use the registers!
      for (uint resIdxM; resIdxMComparison; ++resIdxM) {
        for (uint resIdxN; resIdxNComparison; ++resIdxN) {
          // TODO: compute index to accumulate into
          // again should look p similar to logic found in kernel 1
          threadResults[] += regM[resIdxM] * regN[resIdxN];
        }
      }

    }
    __syncthreads();

  }

  // generalize to C
  // still has to be done in the tile for each thread
  for (uint resIdxM = 0; resIdxMComparison; ++resIdxM) {
    for (uint resIdxN = 0; resIdxNComparison; ++resIdxN) {
      // TODO: determine the correct index for correctly iterating through the whole matrix
      // recall: C[(threadRow * TM + resultIdx) * N + threadCol]
      // how do we incorporate resIdxM/N, TM/TN instead of just resultIdx from the 1d case?
      C[] = alpha * threadResults[] + beta * C[];
    }
  }

}

int main() {
    int M = 4096, N = 4096, K = 4096;

    std::cout << "testing SGEMM with 2D Blocktiling: " << std::endl;

    const uint BK = 8;
    const uint TM = 8;
    const uint TN = 8;
    const uint BM = 128;
    const uint BN = 128;
    dim3 gridDim(CEIL_DIV(N, BN), CEIL_DIV(M, BM));
    dim3 blockDim((BM * BN) / (TM * TN));
    test_sgemm(M, N, K, gridDim, blockDim, sgemm_blocktiling_2d<BM, BN, BK, TM, TN>);

    return 0;
}

In [ ]:
!cp kernel_5.cpp kernel_5.cu
!nvcc kernel_5.cu -o kernel_5 -lcublas
!./kernel_5

In [ ]:
with open('kernel_5.cu', 'r') as file:
    user_kernel_5 = file.read()
check_solution(5, user_kernel_5, M=4096, N=4096, K=4096)

# Kernel 6: Vectorize SMEM and GMEM Accesses


### Introduction

In [ ]:
# we can still push for even more performance!
# if we look at the assembly that our code outputs
# we would notice that loading in data from Bs into regN is a vectorized operation
# so with one instruction, we got multiple pieces of data loaded in
# but this doesn't happen with As, since the memory isn't contiguous
# if we can get As into the same shape as Bs, we should get the vectorized loads
# so we just transpose A when we load it into As, and that should do the trick
#
# syntax tips here -> https://developer.nvidia.com/blog/cuda-pro-tip-increase-performance-with-vectorized-memory-access/

### Code

In [ ]:
%%writefile kernel_6.cpp
#include <iostream>
#include <cmath>
#include <cassert>

#include "test_sgemm3.cu"


template <const int BM, const int BN, const int BK, const int TM, const int TN>
// we remove A and B from being const float *
// to being float *
// because we want to load w vector instructions
__global__ void sgemm_vector(int M, int N, int K, float alpha, float *A, float *B, float beta, float *C) {

  // as before
  const uint cRow = blockIdx.x;
  const uint cCol = blockIdx.y;


  // as before
  const int threadCol = threadIdx.x % (BN / TN);
  const int threadRow = threadIdx.x / (BN / TN);

  // allocate space as before
  __shared__ float As[BM * BK];
  __shared__ float Bs[BK * BN];

  // move pointers as before
  A += cRow * BM * K;
  B += cCol * BN;
  C += cRow * BM * N + cCol * BN;

  // we will load 4 floats at a time
  // this is because we will use the register float4 provided by cuda
  // TODO: make sure we have 4 elements per thread
  const uint innerRowA = ...
  const uint innerColA = ...
  const uint innerRowB = ...
  const uint innerColB = ...

  // as before
  float threadResults[TM * TN] = {0.0};
  float regM[TM] = {0.0};
  float regN[TN] = {0.0};


  // loop over block tiles as before
  for (uint blockIdx = 0; blockIdx < K; blockIdx += BK) {
    // reinterpret_cast allows us to use the vector data type

    // TODO: load B using the vector instructions and float4

    // TODO: transpose A while loading it into the vector data type float4


    __syncthreads();

    // advance blocktile as before
    A += BK;
    B += BK * N;

    // 2d blocktiling loop as before
    for (uint dotIdx = 0; dotIdx < BK; ++dotIdx) {
      // load into registers
      // TODO: rewrite the logic for reading into regM
      // remember how we transposed A when loading into As?
      for (uint i = 0; i < TM; ++i) {
        regM[i] = ...
      }
      for (uint i = 0; i < TN; ++i) {
        regN[i] = Bs[dotIdx * BN + threadCol * TN + i];
      }

      // same as before
      for (uint resIdxM = 0; resIdxM < TM; ++resIdxM) {
        for (uint resIdxN = 0; resIdxN < TN; ++resIdxN) {
          threadResults[resIdxM * TN + resIdxN] +=
              regM[resIdxM] * regN[resIdxN];
        }
      }

    }
    __syncthreads();

  }

  // generalize to C
  // TODO: use vector instructions now

  for (uint resIdxM = 0; resIdxM < TM; resIdxM += 1) {
    // change the increment to abide by using float4
    for (uint resIdxN = 0; resIdxN < TN; resIdxN_increment) {

      // TODO: affine transform while loading into vector

      // write tmp with vector instructions

    }
  }

}

int main() {
    int M = 4096, N = 4096, K = 4096;

    std::cout << "testing SGEMM with 2D Blocktiling and Vectorizing: " << std::endl;

    const uint BK = 8;
    const uint TM = 8;
    const uint TN = 8;
    const uint BM = 128;
    const uint BN = 128;
    dim3 gridDim(CEIL_DIV(N, BN), CEIL_DIV(M, BM));
    dim3 blockDim((BM * BN) / (TM * TN));
    test_sgemm(M, N, K, gridDim, blockDim, sgemm_vector<BM, BN, BK, TM, TN>);

    return 0;
}

In [ ]:
!cp kernel_6.cpp kernel_6.cu
!nvcc kernel_6.cu -o kernel_6 -lcublas
!./kernel_6

In [ ]:
with open('kernel_6.cu', 'r') as file:
    user_kernel_6 = file.read()
check_solution(6, user_kernel_6, M=4096, N=4096, K=4096)